# Pipelines and models

In [221]:
# set up working catalog
import sys
from pathlib import Path
project_path = str(Path().cwd().parent.resolve())
if project_path not in sys.path:
    sys.path.append(project_path)

# imports
from common.utils import get_data

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [222]:
data = get_data()

X = data.drop("Target", axis=1)
y = data["Target"]

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)

data.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,single,2nd phase - general contingent,6th choice,Animation and Multimedia Design,Daytime,Secondary education,122.0,Portuguese,Basic Ed 3rd Cycle,Other - 11th Year,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,single,International student (bachelor),2nd choice,Tourism,Daytime,Secondary education,160.0,Portuguese,Secondary Education - 12th Year or Eq.,Higher Ed - Degree,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,single,1st phase - general contingent,6th choice,Communication Design,Daytime,Secondary education,122.0,Portuguese,Basic Ed 1st Cycle (4th/5th),Basic Ed 1st Cycle (4th/5th),...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,single,2nd phase - general contingent,3rd choice,Journalism and Communication,Daytime,Secondary education,122.0,Portuguese,Basic Ed 2nd Cycle (6th–8th),Basic Ed 1st Cycle (4th/5th),...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,married,Over 23 years old,2nd choice,Social Service (evening attendance),Evening,Secondary education,100.0,Portuguese,Basic Ed 1st Cycle (4th/5th),Basic Ed 2nd Cycle (6th–8th),...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [223]:
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),                  # strategy="mean"
    ("scaler", StandardScaler()), 
])

categoric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),           # strategy="constant"
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, numeric_features),
        ("cat", categoric_pipeline, categorical_features)
    ]
)

In [224]:
def get_accurancy_dataframe(model, X_train_val, y_train_val, cv=5):    
    pipeline = Pipeline([
        ("preprocessing", preprocessor),
        ("classifier", model)
    ])
    
    results = np.empty((0, 3))
    
    for _ in range(cv):
        X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2)
        
        pipeline.fit(X_train, y_train)
        
        y_pred_train = pipeline.predict(X_train)
        y_pred_val = pipeline.predict(X_val)
        y_pred_test = pipeline.predict(X_test)
        
        results = np.vstack([results, np.array([
            accuracy_score(y_train, y_pred_train), 
            accuracy_score(y_val, y_pred_val),
            accuracy_score(y_test, y_pred_test)
        ])])
        
    result_df = pd.DataFrame(results.T, index=["Train", "Validation", "Test"])
    return result_df

In [225]:
get_accurancy_dataframe(LogisticRegression(max_iter=250), X_train_val, y_train_val)

,0,1,2,3,4
Train,0.814906,0.816319,0.815260,0.817379,0.824090
Validation,0.765537,0.781073,0.772599,0.769774,0.747175
Test,0.764972,0.766102,0.758192,0.755932,0.761582


In [226]:
get_accurancy_dataframe(DecisionTreeClassifier(max_depth = 6, min_samples_split=10, min_samples_leaf=2), X_train_val, y_train_val)

,0,1,2,3,4
Train,0.801130,0.794066,0.793712,0.798658,0.787708
Validation,0.731638,0.723164,0.747175,0.741525,0.724576
Test,0.723164,0.719774,0.736723,0.728814,0.712994


In [227]:
get_accurancy_dataframe(SVC(), X_train_val, y_train_val)

,0,1,2,3,4
Train,0.848463,0.847757,0.841046,0.841752,0.843518
Validation,0.764124,0.766949,0.783898,0.781073,0.771186
Test,0.749153,0.741243,0.754802,0.743503,0.746893
